In [1]:
from Datasets import CountceptionPickleDataset, CountceptionRawDataset
from matplotlib import pyplot as plt
from Models import CountCeptionModel
from notebookTrain import *

model = CountCeptionModel()
model = model.load_from_checkpoint("C:/Users/gator/FullerLab/BrainStemSegmenter/lightning_logs/version_35/checkpoints/epoch=999-step=10000.ckpt")
# model.eval()
model = model.to(torch.device("cpu"))

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [1]:
from roifile import ImagejRoi
image_folder = "C:/Users/gator/OneDrive - University of Florida/10x images for quantification/Manual Counts copy/TO TEST CODE/"
files = []
for file_name in os.listdir(image_folder):
    if(".roi" in file_name):
        roi_file_path = image_folder + file_name
        image_path = image_folder + file_name.replace("NEUN.roi", "3C.tiff")
        mask_path = image_folder + file_name.replace("NEUN.roi", "3C Mask.png")
        files.append([image_path, mask_path, roi_file_path])

image_data = []
for file in files:
    image = imread(file[0])
    # image = image[:,:,1] #Make image only NEUN
    tensor_img = torch.stack([transforms.ToTensor()(image)])

    tensor_pred = model.forward(tensor_img)
    pred = tensor_pred[0].detach().permute(1,2,0)

    mask = imread(file[1])
    coords = ImagejRoi.fromfile(file[2]).coordinates()
    image_data.append([image, mask, pred, coords, file[0].split("/")[-1].split(".")[0]])

NameError: name 'imread' is not defined

In [1]:
patch_size=32
truth_alpha = 0.2
truth_color = "blue"
pred_alpha = 0.1
save_dpi = 600

for data in image_data:
    plt.cla()

    for coord in data[3]:
        circle = plt.Circle((coord[0]+int(patch_size/2), coord[1]+int(patch_size/2)), 10 ,alpha=truth_alpha, fill = True , color=truth_color, linewidth=0)
        plt.gca().add_artist(circle)

    NEUN_image = np.pad(data[0][:,:,1]*3,int(patch_size/2), "constant")

    padded_mask = np.pad(data[1][:,:],int(patch_size/2), "constant", constant_values=(0))
    pred_masked = data[2] * np.dstack([padded_mask/255])

    plt.imshow(np.dstack([NEUN_image,NEUN_image,NEUN_image]))
    plt.imshow(np.dstack([np.zeros(pred_masked),pred_masked,np.zeros(pred_masked)]),alpha=pred_alpha)

    ef = ((patch_size / 1) ** 2.0)
    pred_count = (pred_masked / ef).sum(axis=(0, 1)).numpy()[0]
    plt.savefig(data[4]+"_model_output_act_" + str(len(data[3])) + "_pred_" + str(pred_count) + ".png", dpi=save_dpi)
    plt.show()


NameError: name 'image_data' is not defined

In [55]:
offset = 100
patch_size = 32
ef = ((patch_size / 1) ** 2.0)
pred_count = (pred_masked / ef).sum(axis=(0, 1))
print(pred_count)

tensor([182.0380], dtype=torch.float64)
